Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [ ]:
tf.nn.l2_loss2()

### Logistic regression

In [38]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + 0.009*tf.nn.l2_loss(weights)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [39]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 46.288483
Minibatch accuracy: 10.2%
Validation accuracy: 17.4%
Minibatch loss at step 500: 0.883631
Minibatch accuracy: 83.6%
Validation accuracy: 80.9%
Minibatch loss at step 1000: 0.831327
Minibatch accuracy: 83.6%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 0.883039
Minibatch accuracy: 77.3%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 0.843651
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 2500: 0.728958
Minibatch accuracy: 85.9%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 0.777933
Minibatch accuracy: 80.5%
Validation accuracy: 80.7%
Test accuracy: 87.5%


### 1-layer NN


In [54]:
batch_size = 128
num_hidden_nodes = 1024

with tf.device('/cpu:0'):

    graph = tf.Graph()
    with graph.as_default():

        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                            shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        # Variables.
        weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
        biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))

        weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))

        # Training computation.

        vector_after_relu = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
        logits = tf.matmul(vector_after_relu, weights_2) + biases_2
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
        loss += 0.001*tf.nn.l2_loss(weights_1) + 0.001*tf.nn.l2_loss(weights_2)
        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)

        valid_after_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
        valid_prediction = tf.nn.softmax(tf.matmul(valid_after_relu, weights_2) + biases_2)

        test_after_relu = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
        test_prediction = tf.nn.softmax(tf.matmul(test_after_relu, weights_2) + biases_2)

In [55]:
%%time
num_steps = 3001
with tf.device('/gpu:0'):
    with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
        print(tf.global_variables_initializer().run())
        print("Initialized")
        for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

None
Initialized
Minibatch loss at step 0: 599.520081
Minibatch accuracy: 11.7%
Validation accuracy: 31.2%
Minibatch loss at step 500: 194.580383
Minibatch accuracy: 83.6%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 115.694542
Minibatch accuracy: 80.5%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 68.359894
Minibatch accuracy: 82.8%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 41.464222
Minibatch accuracy: 82.8%
Validation accuracy: 84.5%
Minibatch loss at step 2500: 25.157030
Minibatch accuracy: 85.9%
Validation accuracy: 86.7%
Minibatch loss at step 3000: 15.482355
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Test accuracy: 92.8%
CPU times: user 16.8 s, sys: 2.69 s, total: 19.5 s
Wall time: 16.1 s


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [72]:
batch_size = 128
num_hidden_nodes = 1024

with tf.device('/cpu:0'):

    graph = tf.Graph()
    with graph.as_default():

        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                            shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        # Variables.
        weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
        biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))

        weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))

        # Training computation.

        vector_after_relu = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
        
        dropped_vector = tf.nn.dropout(vector_after_relu, keep_prob=0.5)
        logits = tf.matmul(dropped_vector, weights_2) + biases_2
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
        loss += 0.001*tf.nn.l2_loss(weights_1) + 0.001*tf.nn.l2_loss(weights_2)
        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)

        valid_after_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
        valid_prediction = tf.nn.softmax(tf.matmul(valid_after_relu, weights_2) + biases_2)

        test_after_relu = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
        test_prediction = tf.nn.softmax(tf.matmul(test_after_relu, weights_2) + biases_2)

In [73]:
%%time
num_steps = 3001
with tf.device('/gpu:0'):
    with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
        print(tf.global_variables_initializer().run())
        print("Initialized")
        for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
            offset = ((step % batchnum) * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

None
Initialized
Minibatch loss at step 0: 769.166992
Minibatch accuracy: 14.1%
Validation accuracy: 35.7%
Minibatch loss at step 500: 204.187866
Minibatch accuracy: 75.8%
Validation accuracy: 82.0%
Minibatch loss at step 1000: 117.119576
Minibatch accuracy: 78.9%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 70.090179
Minibatch accuracy: 74.2%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 42.369843
Minibatch accuracy: 71.9%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 25.170469
Minibatch accuracy: 85.9%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 15.582275
Minibatch accuracy: 78.1%
Validation accuracy: 85.7%
Test accuracy: 92.0%
CPU times: user 19.1 s, sys: 3.11 s, total: 22.2 s
Wall time: 19.6 s


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [375]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))

    weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.

    vector_after_relu = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)

    dropped_vector = tf.nn.dropout(vector_after_relu, keep_prob=0.5)
    logits = tf.matmul(dropped_vector, weights_2) + biases_2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
        0.0013*tf.nn.l2_loss(weights_1) + 0.0013*tf.nn.l2_loss(weights_2))
 
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    valid_after_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_after_relu, weights_2) + biases_2)

    test_after_relu = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
    test_prediction = tf.nn.softmax(tf.matmul(test_after_relu, weights_2) + biases_2)

In [376]:
%%time
num_steps = 3001
with tf.device('/gpu:0'):
    with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
        print(tf.global_variables_initializer().run())
        print("Initialized")
        for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

None
Initialized
Minibatch loss at step 0: 879.345032
Minibatch accuracy: 11.7%
Validation accuracy: 33.6%
Minibatch loss at step 500: 219.402603
Minibatch accuracy: 73.4%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 112.512520
Minibatch accuracy: 71.9%
Validation accuracy: 82.0%
Minibatch loss at step 1500: 57.376984
Minibatch accuracy: 74.2%
Validation accuracy: 82.9%
Minibatch loss at step 2000: 29.970148
Minibatch accuracy: 78.1%
Validation accuracy: 85.0%
Minibatch loss at step 2500: 15.695998
Minibatch accuracy: 86.7%
Validation accuracy: 86.1%
Minibatch loss at step 3000: 8.508307
Minibatch accuracy: 86.7%
Validation accuracy: 86.3%
Test accuracy: 92.4%
CPU times: user 19.8 s, sys: 3.34 s, total: 23.1 s
Wall time: 20.1 s


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [341]:
batch_size = 128
num_hidden_nodes_1 = 1024
num_hidden_nodes_2 = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes_1], \
                                                stddev=np.sqrt(2.0 /(image_size* image_size))))
    biases_1 = tf.Variable(tf.zeros([num_hidden_nodes_1]))

    weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes_1, num_hidden_nodes_2], \
                                                stddev=np.sqrt(2.0 /num_hidden_nodes_1)))
    biases_2 = tf.Variable(tf.zeros([num_hidden_nodes_2]))
    
    weights_3 = tf.Variable(tf.truncated_normal([num_hidden_nodes_2, num_labels],\
                                                stddev=np.sqrt(2.0 /num_hidden_nodes_2)))
    biases_3 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.

    layer1_out = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    dropped_vector_1 = tf.nn.dropout(layer1_out, keep_prob=0.5)
    
    layer2_out = tf.nn.relu(tf.matmul(dropped_vector_1, weights_2) + biases_2)
    dropped_vector_2 = tf.nn.dropout(layer2_out, keep_prob=0.5)
    
    logits = tf.matmul(dropped_vector_2, weights_3) + biases_3
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
        0.0013*tf.nn.l2_loss(weights_1) + 0.0015*tf.nn.l2_loss(weights_2) + 0.0013*tf.nn.l2_loss(weights_3))
 
    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.0005, global_step, decay_steps=10000, decay_rate=0.7)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)
#    optimizer = tf.train.GradientDescentOptimizer(0.0002).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    valid_layer1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
    valid_layer2 = tf.nn.relu(tf.matmul(valid_layer1, weights_2) + biases_2)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_layer2, weights_3) + biases_3)

    test_layer1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
    test_layer2 = tf.nn.relu(tf.matmul(test_layer1, weights_2) + biases_2)
    test_prediction = tf.nn.softmax(tf.matmul(test_layer2, weights_3) + biases_3)

In [342]:
%%time
%%notify
num_steps = 3001
with tf.device('/gpu:0'):
    with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
        print(tf.global_variables_initializer().run())
        print("Initialized")
        for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

None
Initialized
Minibatch loss at step 0: 4.792294
Minibatch accuracy: 7.0%
Validation accuracy: 36.2%
Minibatch loss at step 500: 1.614178
Minibatch accuracy: 84.4%
Validation accuracy: 84.7%
Minibatch loss at step 1000: 1.093725
Minibatch accuracy: 82.0%
Validation accuracy: 85.8%
Minibatch loss at step 1500: 0.946629
Minibatch accuracy: 78.9%
Validation accuracy: 86.7%
Minibatch loss at step 2000: 0.676985
Minibatch accuracy: 85.9%
Validation accuracy: 86.9%
Minibatch loss at step 2500: 0.615708
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 3000: 0.667090
Minibatch accuracy: 85.9%
Validation accuracy: 86.9%
Test accuracy: 93.1%


<IPython.core.display.Javascript object>

CPU times: user 41.7 s, sys: 8.5 s, total: 50.2 s
Wall time: 45 s


In [371]:
batch_size = 128
num_hidden_nodes_1 = 2048
num_hidden_nodes_2 = 1024
num_hidden_nodes_3 = 512


graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes_1], 
                           stddev=np.sqrt(2.0 /(image_size* image_size))))
    biases_1 = tf.Variable(tf.zeros([num_hidden_nodes_1]))

    weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes_1, num_hidden_nodes_2], 
                            stddev=np.sqrt(2.0 /(image_size* image_size))))
    biases_2 = tf.Variable(tf.zeros([num_hidden_nodes_2]))
    
    weights_3 = tf.Variable(tf.truncated_normal([num_hidden_nodes_2, num_hidden_nodes_3], 
                            stddev=np.sqrt(2.0 /(image_size* image_size))))
    biases_3 = tf.Variable(tf.zeros([num_hidden_nodes_3]))
    
    weights_4 = tf.Variable(tf.truncated_normal([num_hidden_nodes_3, num_labels], 
                                                stddev=np.sqrt(2.0 /(image_size* image_size))))
    biases_4 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.

    layer1_out = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    dropped_vector_1 = tf.nn.dropout(layer1_out, keep_prob=0.5)
    
    layer2_out = tf.nn.relu(tf.matmul(dropped_vector_1, weights_2) + biases_2)
    dropped_vector_2 = tf.nn.dropout(layer2_out, keep_prob=0.5)
    
    layer3_out = tf.nn.relu(tf.matmul(dropped_vector_2, weights_3) + biases_3)
    dropped_vector_3 = tf.nn.dropout(layer3_out, keep_prob=0.5)
    
    logits = tf.matmul(dropped_vector_3, weights_4) + biases_4
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
        0.0013*tf.nn.l2_loss(weights_1) + 0.0013*tf.nn.l2_loss(weights_2) + 0.0013*tf.nn.l2_loss(weights_3) + 
        0.0013*tf.nn.l2_loss(weights_4))
 
    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.0003, global_step, decay_steps=10000, decay_rate=0.1)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)
#    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    valid_layer1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
    valid_layer2 = tf.nn.relu(tf.matmul(valid_layer1, weights_2) + biases_2)
    valid_layer3 = tf.nn.relu(tf.matmul(valid_layer2, weights_3) + biases_3)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_layer3, weights_4) + biases_4)

    test_layer1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
    test_layer2 = tf.nn.relu(tf.matmul(test_layer1, weights_2) + biases_2)
    test_layer3 = tf.nn.relu(tf.matmul(test_layer2, weights_3) + biases_3)
    test_prediction = tf.nn.softmax(tf.matmul(test_layer3, weights_4) + biases_4)

In [372]:
%%time
%%notify
num_steps = 3001
with tf.device('/gpu:0'):
    with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
        print(tf.global_variables_initializer().run())
        print("Initialized")
        for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

None
Initialized
Minibatch loss at step 0: 8.708662
Minibatch accuracy: 10.2%
Validation accuracy: 17.1%
Minibatch loss at step 500: 4.487982
Minibatch accuracy: 82.8%
Validation accuracy: 84.1%
Minibatch loss at step 1000: 3.287244
Minibatch accuracy: 83.6%
Validation accuracy: 85.1%
Minibatch loss at step 1500: 2.463896
Minibatch accuracy: 82.0%
Validation accuracy: 86.2%
Minibatch loss at step 2000: 1.803397
Minibatch accuracy: 82.8%
Validation accuracy: 86.5%
Minibatch loss at step 2500: 1.352890
Minibatch accuracy: 88.3%
Validation accuracy: 87.2%
Minibatch loss at step 3000: 1.125922
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Test accuracy: 93.3%


<IPython.core.display.Javascript object>

CPU times: user 1min 18s, sys: 17 s, total: 1min 35s
Wall time: 1min 35s
